## Test of cbpro library

In [11]:
import pandas as pd
from datetime import date
import cbpro
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

In [2]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [37]:
#Download data
public_client = cbpro.PublicClient()

BTC_USDT1 = public_client.get_product_historic_rates('BTC-USDT', '2023-01-01', '2023-06-01', 86400)
ETH_USDT1 = public_client.get_product_historic_rates('ETH-USDT', '2023-01-01', '2023-06-01', 86400)
BTC_USDT2 = public_client.get_product_historic_rates('BTC-USDT', '2022-07-01', '2022-12-31', 86400)
ETH_USDT2 = public_client.get_product_historic_rates('ETH-USDT', '2022-07-01', '2022-12-31', 86400)
BTC_USDT3 = public_client.get_product_historic_rates('BTC-USDT', '2022-01-01', '2022-06-30', 86400)
ETH_USDT3 = public_client.get_product_historic_rates('ETH-USDT', '2022-01-01', '2022-06-30', 86400)
BTC_USDT = BTC_USDT1 + BTC_USDT2 + BTC_USDT3
ETH_USDT = ETH_USDT1 + ETH_USDT2 + ETH_USDT3

In [38]:
# Rename columns
df_hist_data= pd.DataFrame(BTC_USDT)
df_hist_data.columns = [ 'date', 'BTC_low', 'BTC_high', 'BTC_open', 'BTC_close', 'BTC_volume' ]
df_hist_data['date']= df_hist_data['date'].apply(lambda x: date.fromtimestamp(x))
coin_res = [ETH_USDT]
coin_names = ['ETH']
for coin, name in zip (coin_res, coin_names):
    tmp = pd.DataFrame(coin)
    tmp.columns = [ 'date', 'low', 'high', 'open', 'close', 'volume' ]
    df_hist_data[f'{name}_close'] = tmp['close']

df_hist_data.sort_values(by=['date'], inplace=True)

In [ ]:
# Plot data we got
fig, ax = plt.subplots(1, 1, sharex = True)
fig.set_figheight(11)
fig.set_figwidth(8.5)
ax.plot(df_hist_data['date'], df_hist_data['BTC_close'], color = 'black', label = 'Historic price')
for tick in ax.get_xticklabels():
        tick.set_rotation(45)

In [39]:
scaler  = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(np.array(df_hist_data['BTC_close'].values).reshape(-1,1) )

In [40]:
prediction_days = 60
x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x,0])
x_train, y_train = (np.array(x_train), np.array(y_train))
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [46]:
model = Sequential()
model.add(LSTM(units = 50, activation='tanh', return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(units = 1))
optimizer = Adam(learning_rate=0.001, name='Adam')

model.compile(optimizer = optimizer, loss = 'mse', metrics=['accuracy'])
model.summary()

2024-02-20 13:49:49.521736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-02-20 13:49:49.524669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-02-20 13:49:49.527520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 50)            10400     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_7 (Dropout)         (None, 60, 50)            0         
                                                                 
 dense_2 (Dense)             (None, 60, 1)             51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


2024-02-20 13:49:49.809357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-02-20 13:49:49.811534: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-02-20 13:49:49.813440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
model.fit(x_train, y_train, epochs=25, batch_size=8)

In [59]:
test_data = public_client.get_product_historic_rates('BTC-USDT', '2023-06-02', '2023-12-01', 86400)
test_data= pd.DataFrame(test_data)
test_data.columns = [ 'date', 'BTC_low', 'BTC_high', 'BTC_open', 'BTC_close', 'BTC_volume' ]
test_data['date']= test_data['date'].apply(lambda x: date.fromtimestamp(x))
test_data.sort_values(by=['date'], inplace=True)

In [69]:
total_dataset = pd.concat((df_hist_data[['date', 'BTC_close']], test_data[['date', 'BTC_close']]), axis = 0)
model_inputs = total_dataset[len(total_dataset)-len(test_data)-prediction_days:]['BTC_close'].values
len(model_inputs)

243

In [70]:
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.fit_transform(model_inputs)
model_inputs

array([[0.19796902],
       [0.22520803],
       [0.22520435],
       [0.21503473],
       [0.20605449],
       [0.20848329],
       [0.23664949],
       [0.33348283],
       [0.37500525],
       [0.35198167],
       [0.38792402],
       [0.39395111],
       [0.38173049],
       [0.38227178],
       [0.31823836],
       [0.38767731],
       [0.27139363],
       [0.23078883],
       [0.15829951],
       [0.19922246],
       [0.18198074],
       [0.17646327],
       [0.23527822],
       [0.2429417 ],
       [0.32205537],
       [0.3093752 ],
       [0.30322586],
       [0.30125439],
       [0.21762114],
       [0.26181981],
       [0.28833562],
       [0.2749249 ],
       [0.32388323],
       [0.27522758],
       [0.2443056 ],
       [0.18803287],
       [0.18568139],
       [0.18271571],
       [0.1367967 ],
       [0.12352222],
       [0.12240577],
       [0.13303566],
       [0.15092178],
       [0.1415166 ],
       [0.16874529],
       [0.12587297],
       [0.13045588],
       [0.146